In [12]:
#importing the libraries needed 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import re

In [13]:
#Declaring the headers 
headers = {"Accept-Language": "en-US,en;q=0.5"}

In [14]:
#declaring the list of empty variables, So that we can append the data overall

movie_name = []
year = []
certificate = []
time=[]
genre = []
rating=[]
metascore =[]
votes = []
gross = []
description = []

In [15]:
#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,1000,100)

In [16]:
#the whole core of the script
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt", headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year.append(re.findall("\d+", year_of_release)[0])
        
        certificate_rating = store.p.find("span", class_ = 'certificate').text if store.p.find("span", class_ = 'certificate') else np.nan
        certificate.append(certificate_rating)
        
        runtime = store.p.find("span", class_ = 'runtime').text if store.p.find("span", class_ = 'runtime') else np.nan
        runtime = np.nan if runtime is np.nan else runtime[:len(runtime) - 4]
        time.append(runtime)
        
        movie_genre = store.p.find("span", class_ = 'genre').text.replace('\n', '') if store.p.find("span", class_ = 'genre') else np.nan
        genre.append(movie_genre)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '')
        rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else np.nan
        metascore.append(meta)
        
        
        value = store.find_all('span', attrs = {'name': "nv"})
        
        vote = value[0].text.replace(',', '')
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 and re.match("\$(\d+?.?\d+?)M", value[1].text) else np.nan
        grosses = np.nan if grosses is np.nan else grosses[1:len(grosses) - 1]
        gross.append(grosses)
        
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else np.nan
        description.append(description_)

In [17]:
#creating a dataframe 
movie_list = pd.DataFrame({ "Movie_Name": movie_name, "Year_of_Release" : year, "C_Rating": certificate,  "Watch_Time_(min)": time, "Genre": genre , "Movie_Rating": rating, "Metascore": metascore, "Votes" : votes, "Gross_($M)": gross, "Description": description  })

In [18]:
movie_list.head(20)

,Movie_Name,Year_of_Release,C_Rating,Watch_Time_(min),Genre,Movie_Rating,Metascore,Votes,Gross_($M),Description
0,The Shawshank Redemption,1994,R,142,Drama,9.3,82,2735235,28.34,"Over the course of several years, two convicts..."
1,The Godfather,1972,R,175,"Crime, Drama",9.2,100,1902109,134.97,The aging patriarch of an organized crime dyna...
2,The Dark Knight,2008,PG-13,152,"Action, Crime, Drama",9.0,84,2708419,534.86,When the menace known as the Joker wreaks havo...
3,Schindler's List,1993,R,195,"Biography, Drama, History",9.0,95,1380843,96.90,"In German-occupied Poland during World War II,..."
4,The Lord of the Rings: The Return of the King,2003,PG-13,201,"Action, Adventure, Drama",9.0,94,1881383,377.85,Gandalf and Aragorn lead the World of Men agai...
5,12 Angry Men,1957,Approved,96,"Crime, Drama",9.0,97,809518,4.36,The jury in a New York City murder trial is fr...
6,The Godfather Part II,1974,R,202,"Crime, Drama",9.0,90,1296679,57.30,The early life and career of Vito Corleone in ...
7,Pulp Fiction,1994,R,154,"Crime, Drama",8.9,95,2101929,107.93,"The lives of two mob hitmen, a boxer, a gangst..."
8,Inception,2010,PG-13,148,"Action, Adventure, Sci-Fi",8.8,74,2404179,292.58,A thief who steals corporate secrets through t...
9,Fight Club,1999,R,139,Drama,8.8,66,2177808,37.03,An insomniac office worker and a devil-may-car...


In [19]:
movie_list.to_csv("clean data\Top 1000 IMDb movies.csv")

In [20]:
movie_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Movie_Name        1000 non-null   object
 1   Year_of_Release   1000 non-null   object
 2   C_Rating          984 non-null    object
 3   Watch_Time_(min)  1000 non-null   object
 4   Genre             1000 non-null   object
 5   Movie_Rating      1000 non-null   object
 6   Metascore         844 non-null    object
 7   Votes             1000 non-null   object
 8   Gross_($M)        814 non-null    object
 9   Description       1000 non-null   object
dtypes: object(10)
memory usage: 78.2+ KB


In [21]:
movie_list.isnull().sum()

Movie_Name            0
Year_of_Release       0
C_Rating             16
Watch_Time_(min)      0
Genre                 0
Movie_Rating          0
Metascore           156
Votes                 0
Gross_($M)          186
Description           0
dtype: int64

In [22]:
#declaring the list of empty variables, So that we can append the data overall

movie_name = []
year = []
certificate = []
time=[]
genre = []
rating=[]
metascore =[]
votes = []
gross = []
description = []

In [23]:
#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,10000,100)

In [24]:
#the whole core of the script
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt", headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year.append(re.findall("\d+", year_of_release)[0])
        
        certificate_rating = store.p.find("span", class_ = 'certificate').text if store.p.find("span", class_ = 'certificate') else np.nan
        certificate.append(certificate_rating)
        
        runtime = store.p.find("span", class_ = 'runtime').text if store.p.find("span", class_ = 'runtime') else np.nan
        runtime = np.nan if runtime is np.nan else runtime[:len(runtime) - 4]
        time.append(runtime)
        
        movie_genre = store.p.find("span", class_ = 'genre').text.replace('\n', '') if store.p.find("span", class_ = 'genre') else np.nan
        genre.append(movie_genre)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '')
        rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else np.nan
        metascore.append(meta)
        
        
        value = store.find_all('span', attrs = {'name': "nv"})
        
        vote = value[0].text.replace(',', '')
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 and re.match("\$(\d+?.?\d+?)M", value[1].text) else np.nan
        grosses = np.nan if grosses is np.nan else grosses[1:len(grosses) - 1]
        gross.append(grosses)
        
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else np.nan
        description.append(description_)

In [25]:
#creating a dataframe 
movie_list = pd.DataFrame({ "Movie_Name": movie_name, "Year_of_Release" : year, "C_Rating": certificate,  "Watch_Time_(min)": time, "Genre": genre , "Movie_Rating": rating, "Metascore": metascore, "Votes" : votes, "Gross_($M)": gross, "Description": description  })

In [26]:
movie_list.head(20)

,Movie_Name,Year_of_Release,C_Rating,Watch_Time_(min),Genre,Movie_Rating,Metascore,Votes,Gross_($M),Description
0,The Shawshank Redemption,1994,R,142,Drama,9.3,82,2735235,28.34,"Over the course of several years, two convicts..."
1,The Godfather,1972,R,175,"Crime, Drama",9.2,100,1902109,134.97,The aging patriarch of an organized crime dyna...
2,The Chaos Class,1975,NaN,87,"Comedy, Drama",9.2,NaN,41631,NaN,"Lazy, uneducated students share a very close b..."
3,Daman,2022,NaN,121,"Adventure, Drama",9.1,NaN,13199,NaN,"The film is set in 2015. Sid, is a young docto..."
4,The Dark Knight,2008,PG-13,152,"Action, Crime, Drama",9.0,84,2708419,534.86,When the menace known as the Joker wreaks havo...
5,Schindler's List,1993,R,195,"Biography, Drama, History",9.0,95,1380843,96.90,"In German-occupied Poland during World War II,..."
6,The Lord of the Rings: The Return of the King,2003,PG-13,201,"Action, Adventure, Drama",9.0,94,1881383,377.85,Gandalf and Aragorn lead the World of Men agai...
7,12 Angry Men,1957,Approved,96,"Crime, Drama",9.0,97,809518,4.36,The jury in a New York City murder trial is fr...
8,The Godfather Part II,1974,R,202,"Crime, Drama",9.0,90,1296679,57.30,The early life and career of Vito Corleone in ...
9,Mirror Game,2016,NaN,147,"Crime, Mystery, Thriller",9.0,NaN,27497,NaN,Ayna is an actor and the prison is his stage. ...


In [27]:
movie_list.to_csv("clean data\Top 10000 IMDb movies.csv")

In [28]:
movie_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Movie_Name        9999 non-null   object
 1   Year_of_Release   9999 non-null   object
 2   C_Rating          9645 non-null   object
 3   Watch_Time_(min)  9999 non-null   object
 4   Genre             9999 non-null   object
 5   Movie_Rating      9999 non-null   object
 6   Metascore         7999 non-null   object
 7   Votes             9999 non-null   object
 8   Gross_($M)        7203 non-null   object
 9   Description       9999 non-null   object
dtypes: object(10)
memory usage: 781.3+ KB


In [29]:
movie_list.isnull().sum()

Movie_Name             0
Year_of_Release        0
C_Rating             354
Watch_Time_(min)       0
Genre                  0
Movie_Rating           0
Metascore           2000
Votes                  0
Gross_($M)          2796
Description            0
dtype: int64